In [2]:
from langchain_community.graphs import OntotextGraphDBGraph

# feeding the schema using a user construct query

graph = OntotextGraphDBGraph(
    query_endpoint="http://localhost:7200/repositories/papers",
    query_ontology="CONSTRUCT {?s ?p ?o} FROM <http://cinecakg.com/graph> WHERE {?s ?p ?o}",
)

In [4]:
graph.get_schema

'@prefix dcterms: <http://purl.org/dc/terms/> .\n@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n\n<https://cineca.it/graph#paper1> a <http://schema.org/ScholarlyArticle> ;\n    rdfs:label "Attention Is All You Need" ;\n    dcterms:creator <https://cineca.it/graph#author1>,\n        <https://cineca.it/graph#author2>,\n        <https://cineca.it/graph#author3>,\n        <https://cineca.it/graph#author4>,\n        <https://cineca.it/graph#author5>,\n        <https://cineca.it/graph#author6>,\n        <https://cineca.it/graph#author7>,\n        <https://cineca.it/graph#author8> .\n\n<https://cineca.it/graph#paper2> a <http://schema.org/ScholarlyArticle> ;\n    rdfs:label "The Efficiency Misnomer" ;\n    dcterms:creator <https://cineca.it/graph#author10>,\n        <https://cineca.it/graph#author11>,\n        <https://cineca.it/graph#author12>,\n        <https://cineca.it/graph#author13>,\n        <https://cineca.it/graph#author9> .\n\n

In [5]:
import os

from langchain.chains import OntotextGraphDBQAChain
from langchain_openai import ChatOpenAI

# We'll be using an OpenAI model which requires an OpenAI API Key.
# However, other models are available as well:
# https://python.langchain.com/docs/integrations/chat/

# Set the environment variable `OPENAI_API_KEY` to your OpenAI API key
# os.environ["OPENAI_API_KEY"] = "sk-***"

# Any available OpenAI model can be used here.
# We use 'gpt-4-1106-preview' because of the bigger context window.
# The 'gpt-4-1106-preview' model_name will deprecate in the future and will change to 'gpt-4-turbo' or similar,
# so be sure to consult with the OpenAI API https://platform.openai.com/docs/models for the correct naming.

chain = OntotextGraphDBQAChain.from_llm(
    ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview"),
    graph=graph,
    verbose=True,
)

In [6]:
chain.invoke({chain.input_key: "Who wrote the paper 'Attention is all you need'?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?authorLabel
WHERE {
  ?paper rdfs:label "Attention Is All You Need" .
  ?paper dcterms:creator ?author .
  ?author rdfs:label ?authorLabel .
}

> Finished chain.


"The paper 'Attention is all you need' was authored by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin."

In [7]:
chain.invoke({chain.input_key: "Which researchers from Google Research authored both 'The efficiency misnomer' and 'Attention is all you need'?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?researcherName
WHERE {
  ?paper1 a schema:ScholarlyArticle ;
          rdfs:label "Attention Is All You Need" ;
          dcterms:creator ?author1 .
  ?paper2 a schema:ScholarlyArticle ;
          rdfs:label "The Efficiency Misnomer" ;
          dcterms:creator ?author2 .
  ?author1 rdfs:label ?researcherName ;
           foaf:memberOf <https://cineca.it/graph#organization2> .
  ?author2 rdfs:label ?researcherName ;
           foaf:memberOf <https://cineca.it/graph#organization2> .
}

> Finished chain.


"Based on the information provided, I do not have the data to identify which researchers from Google Research authored both 'The efficiency misnomer' and 'Attention is all you need'. If you can provide the results of a relevant SPARQL query or any additional details, I would be happy to assist you further."

In [8]:
chain.invoke({chain.input_key: "Which researchers worked together on academic papers?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT ?researcher1 ?researcher2 ?paperLabel
WHERE {
  ?paper a schema:ScholarlyArticle ;
         rdfs:label ?paperLabel ;
         dcterms:creator ?author1, ?author2 .
  ?author1 rdfs:label ?researcher1 .
  ?author2 rdfs:label ?researcher2 .
  FILTER (?author1 != ?author2)
}

> Finished chain.


'The researchers who collaborated on academic papers are as follows:\n\n- Ashish Vaswani worked with Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin on the paper titled "Attention Is All You Need".\n- Noam Shazeer collaborated with Ashish Vaswani, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin on the same paper.\n- Niki Parmar co-authored with Ashish Vaswani, Noam Shazeer, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin on "Attention Is All You Need".\n- Jakob Uszkoreit worked alongside Ashish Vaswani, Noam Shazeer, Niki Parmar, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin on the paper.\n- Llion Jones collaborated with Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin on "Attention Is All You Need".\n- Aidan N. Gomez co-authored with Ashish Vaswani, Noam Shazeer, N

In [9]:
chain.invoke({chain.input_key: "List all people from Google Research"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT ?person ?personLabel
WHERE {
  ?person a schema:Person ;
          rdfs:label ?personLabel ;
          foaf:memberOf <https://cineca.it/graph#organization2> .
}

> Finished chain.


'Based on the information provided, I cannot confirm whether any of the individuals listed are associated with Google Research, as the data does not include any details about their affiliations.'

In [10]:
chain.invoke({chain.input_key: "Which researchers from Google Research and from Google Brain collaborated on 'Attention is all you need'?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?researcherName
WHERE {
  ?paper a schema:ScholarlyArticle ;
         rdfs:label "Attention Is All You Need" ;
         dcterms:creator ?author .
  ?author a schema:Person ;
          rdfs:label ?researcherName ;
          foaf:memberOf ?organization .
  ?organization a schema:Organization ;
                rdfs:label ?organizationName .
  FILTER (?organizationName IN ("Google Research", "Google Brain"))
}

> Finished chain.


"The researchers from Google Research and Google Brain who collaborated on the paper 'Attention is All You Need' include Niki Parmar, Jakob Uszkoreit, Llion Jones, Ashish Vaswani, Noam Shazeer, and Łukasz Kaiser."

quali sono i ricercatori dell uni bologna che hanno lavorato con ricercatori del canada

In [12]:
chain.invoke({chain.input_key: "Which researchers from Google Research and from Google Brain collaborated on any papers?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?researcherName
WHERE {
  ?paper a schema:ScholarlyArticle ;
         dcterms:creator ?researcher .
  ?researcher a schema:Person ;
              rdfs:label ?researcherName ;
              foaf:memberOf ?organization .
  ?organization a schema:Organization ;
                rdfs:label ?organizationName .
  FILTER (?organizationName IN ("Google Research", "Google Brain"))
}

> Finished chain.


"Based on the information provided, I can confirm that the following researchers have collaborated on papers: Niki Parmar, Jakob Uszkoreit, Llion Jones, Mostafa Dehghani, Anurag Arnab, Lucas Beyer, Ashish Vaswani, Yi Tay, Noam Shazeer, and Łukasz Kaiser. However, the information does not specify which of these individuals are from Google Research or Google Brain, nor does it detail the specific papers they collaborated on. If you need information about their affiliations or the papers they worked on, I'm unable to provide that with the current data available."

In [14]:
chain.invoke({chain.input_key: "Are there any researchers who worked on more than 1 paper?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX schema: <http://schema.org/>

SELECT ?author ?authorLabel (COUNT(?paper) AS ?papersCount)
WHERE {
  ?paper a schema:ScholarlyArticle ;
         dcterms:creator ?author .
  ?author a schema:Person ;
          rdfs:label ?authorLabel .
}
GROUP BY ?author ?authorLabel
HAVING (COUNT(?paper) > 1)

> Finished chain.


'Based on the information provided, I do not have any data regarding researchers or their publications. Therefore, I cannot confirm whether there are any researchers who have worked on more than one paper. If you have access to specific data or a database, I could assist you in formulating a query to find out.'

In [15]:
chain.invoke({chain.input_key: "How many authors of papers are there?"})[chain.output_key]



> Entering new OntotextGraphDBQAChain chain...
Generated SPARQL:
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX schema: <http://schema.org/>

SELECT (COUNT(DISTINCT ?author) AS ?numberOfAuthors)
WHERE {
  ?paper a schema:ScholarlyArticle ;
         dcterms:creator ?author .
}

> Finished chain.


'Based on the data retrieved from the SPARQL query, there are 13 authors of papers.'